In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from prophet import Prophet
from prophet.plot import plot_plotly
import warnings
warnings.filterwarnings("ignore")

In [71]:
#Units: Passengers in Thousand
#Resources: 한국철도공사 및 (주)SR에서 제공하는「KTX, SRT 여객 수송 통계」

In [72]:
data = pd.read_excel('data.xls')
data

,Unnamed: 0,200404월,200405월,200406월,200407월,200408월,200409월,200410월,200411월,200412월,...,202105월,202106월,202107월,202108월,202109월,202110월,202111월,202112월,202201월,202202월
0,고속철도 여객수 계,"2,127","2,314","1,968","2,152","2,247","2,255","2,278","2,200","2,340",...,"5,910","5,729","5,071","5,024","5,130","6,427","6,866","6,375","5,798","4,959"
1,KTX이용률,61,65,57,61,65,65,65,66,65,...,70,71,59,57,60,74,85,74,67,62
2,SRT이용률,-,-,-,-,-,-,-,-,-,...,106,108,96,96,94,114,120,113,102,99
3,경부선 KTX여객수,"1,771","1,945","1,695","1,845","1,893","1,825","1,912","1,838","1,974",...,"2,102","2,060","1,770","1,695","1,799","2,292","2,521","2,280","2,079","1,754"
4,경부선 KTX이용률,70,75,67,72,75,73,76,76,75,...,66,68,56,53,58,71,82,72,65,60
5,경부선 SRT여객수,-,-,-,-,-,-,-,-,-,...,"1,292","1,249","1,158","1,132","1,107","1,365","1,382","1,356","1,258","1,111"
6,경부선 SRT이용률,-,-,-,-,-,-,-,-,-,...,112,116,102,102,100,121,128,120,108,105
7,호남선 KTX여객수,356,369,273,307,354,430,366,363,366,...,701,673,578,586,611,756,821,743,662,553
8,호남선 KTX이용률,38,39,30,32,37,45,38,39,37,...,66,62,52,51,55,66,74,65,59,53
9,호남선 SRT여객수,-,-,-,-,-,-,-,-,-,...,508,495,446,452,438,537,553,534,483,419


In [73]:
init = True
my_list = []
for c in data:
    if init == True:
        init = False
    else:
        temp_list = data[c].values
        for i in range(len(temp_list)):
            if temp_list[i] == '-':
                temp_list[i] = 0
        my_list.append(temp_list)

In [74]:
my_list = np.array(my_list)
rows = len(my_list)
cols = len(my_list[0])

In [75]:
for i in range(rows):
    for j in range(cols):
        my_list[i][j] = str(my_list[i][j])

In [76]:
for i in range(rows):
    for j in range(cols):
        remove_comma = my_list[i][j].replace(',','')
        f = float(remove_comma)
        my_list[i][j] = f

In [77]:
df1 = {}
df2 = {}
year = 2004
x = []
for m in range(4,13):
    if m < 10:
        x.append(str(0)+str(m)+'-01-'+str(year))
    else:
        x.append(str(m)+'-01-'+str(year))
year +=1
for y in range(18):
    for m in range(1,13):
        if m < 10:
            x.append(str(0)+str(m)+'-01-'+str(year))
        else:
            x.append(str(m)+'-01-'+str(year))
    year +=1

In [78]:
x = x[:-10]
len(x)

215

In [79]:
df1['Time'] = x
df1['Time'] = pd.to_datetime(df1['Time'])
time_series1 = pd.DataFrame(df1)
forecast1 = time_series1
time_series1 = time_series1.set_index('Time')

In [80]:
df2['Time'] = x
df2['Time'] = pd.to_datetime(df2['Time'])
time_series2 = pd.DataFrame(df2)
forecast2 = time_series2
time_series2 = time_series2.set_index('Time')

### Korean High-speed Train Lines

In [81]:
ktx = 'KTX'
srt = 'SRT'
gyeongbu = 'Gyeongbu'
honam = 'Honam'
gyeongjeon = 'Gyeongjeon'
jeolla = 'Jeolla'
donghae = 'Donghae'

lines = [gyeongbu,honam,gyeongjeon,jeolla,donghae]
passengers = 'Passengers'
utility = 'Utility Rate'

In [82]:
total_list = ['Total Passengers']
utility_list = ['KTX Utility Rate','SRT Utility Rate']
concat = total_list + utility_list

for line in lines:
    if line == gyeongbu or line == honam:
        ktx_add = ktx + ' ' + line + ' ' + passengers
        total_list.append(ktx_add)
        concat.append(ktx_add)
        ktx_add = ktx + ' ' + line + ' ' + utility
        utility_list.append(ktx_add)
        concat.append(ktx_add)
        srt_add = srt + ' ' + line + ' ' + passengers
        total_list.append(srt_add)
        concat.append(srt_add)
        srt_add = srt + ' ' + line + ' ' + utility
        utility_list.append(srt_add)
        concat.append(srt_add)
    else:
        ktx_add = ktx + ' ' + line + ' ' + passengers
        total_list.append(ktx_add)
        concat.append(ktx_add)
        ktx_add = ktx + ' ' + line + ' ' + utility
        utility_list.append(ktx_add)
        concat.append(ktx_add)

In [83]:
total_list

['Total Passengers',
 'KTX Gyeongbu Passengers',
 'SRT Gyeongbu Passengers',
 'KTX Honam Passengers',
 'SRT Honam Passengers',
 'KTX Gyeongjeon Passengers',
 'KTX Jeolla Passengers',
 'KTX Donghae Passengers']

In [84]:
utility_list

['KTX Utility Rate',
 'SRT Utility Rate',
 'KTX Gyeongbu Utility Rate',
 'SRT Gyeongbu Utility Rate',
 'KTX Honam Utility Rate',
 'SRT Honam Utility Rate',
 'KTX Gyeongjeon Utility Rate',
 'KTX Jeolla Utility Rate',
 'KTX Donghae Utility Rate']

In [85]:
concat

['Total Passengers',
 'KTX Utility Rate',
 'SRT Utility Rate',
 'KTX Gyeongbu Passengers',
 'KTX Gyeongbu Utility Rate',
 'SRT Gyeongbu Passengers',
 'SRT Gyeongbu Utility Rate',
 'KTX Honam Passengers',
 'KTX Honam Utility Rate',
 'SRT Honam Passengers',
 'SRT Honam Utility Rate',
 'KTX Gyeongjeon Passengers',
 'KTX Gyeongjeon Utility Rate',
 'KTX Jeolla Passengers',
 'KTX Jeolla Utility Rate',
 'KTX Donghae Passengers',
 'KTX Donghae Utility Rate']

In [86]:
count = 0
for c in concat:
    temp = c[-4:len(c)]
    if temp == 'gers':
        time_series1[c] = np.array(list(my_list[:,count]))
        forecast1[c] = np.array(list(my_list[:,count]))
    elif temp == 'Rate':
        time_series2[c] = np.array(list(my_list[:,count]))
        forecast2[c] = np.array(list(my_list[:,count]))
    count +=1

In [87]:
time_series1

,Total Passengers,KTX Gyeongbu Passengers,SRT Gyeongbu Passengers,KTX Honam Passengers,SRT Honam Passengers,KTX Gyeongjeon Passengers,KTX Jeolla Passengers,KTX Donghae Passengers
Time,,,,,,,,
2004-04-01,2127.0,1771.0,0.0,356.0,0.0,0.0,0.0,0.0
2004-05-01,2314.0,1945.0,0.0,369.0,0.0,0.0,0.0,0.0
2004-06-01,1968.0,1695.0,0.0,273.0,0.0,0.0,0.0,0.0
2004-07-01,2152.0,1845.0,0.0,307.0,0.0,0.0,0.0,0.0
2004-08-01,2247.0,1893.0,0.0,354.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2021-10-01,6427.0,2292.0,1365.0,756.0,537.0,507.0,554.0,416.0
2021-11-01,6866.0,2521.0,1382.0,821.0,553.0,546.0,586.0,457.0
2021-12-01,6375.0,2280.0,1356.0,743.0,534.0,501.0,543.0,418.0


In [88]:
time_series1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 215 entries, 2004-04-01 to 2022-02-01
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Total Passengers           215 non-null    float64
 1   KTX Gyeongbu Passengers    215 non-null    float64
 2   SRT Gyeongbu Passengers    215 non-null    float64
 3   KTX Honam Passengers       215 non-null    float64
 4   SRT Honam Passengers       215 non-null    float64
 5   KTX Gyeongjeon Passengers  215 non-null    float64
 6   KTX Jeolla Passengers      215 non-null    float64
 7   KTX Donghae Passengers     215 non-null    float64
dtypes: float64(8)
memory usage: 15.1 KB


In [89]:
time_series2

,KTX Utility Rate,SRT Utility Rate,KTX Gyeongbu Utility Rate,SRT Gyeongbu Utility Rate,KTX Honam Utility Rate,SRT Honam Utility Rate,KTX Gyeongjeon Utility Rate,KTX Jeolla Utility Rate,KTX Donghae Utility Rate
Time,,,,,,,,,
2004-04-01,61.0,0.0,70.0,0.0,38.0,0.0,0.0,0.0,0.0
2004-05-01,65.0,0.0,75.0,0.0,39.0,0.0,0.0,0.0,0.0
2004-06-01,57.0,0.0,67.0,0.0,30.0,0.0,0.0,0.0,0.0
2004-07-01,61.0,0.0,72.0,0.0,32.0,0.0,0.0,0.0,0.0
2004-08-01,65.0,0.0,75.0,0.0,37.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-10-01,74.0,114.0,71.0,121.0,66.0,100.0,86.0,84.0,85.0
2021-11-01,85.0,120.0,82.0,128.0,74.0,106.0,97.0,93.0,96.0
2021-12-01,74.0,113.0,72.0,120.0,65.0,98.0,86.0,83.0,85.0


In [90]:
time_series2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 215 entries, 2004-04-01 to 2022-02-01
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   KTX Utility Rate             215 non-null    float64
 1   SRT Utility Rate             215 non-null    float64
 2   KTX Gyeongbu Utility Rate    215 non-null    float64
 3   SRT Gyeongbu Utility Rate    215 non-null    float64
 4   KTX Honam Utility Rate       215 non-null    float64
 5   SRT Honam Utility Rate       215 non-null    float64
 6   KTX Gyeongjeon Utility Rate  215 non-null    float64
 7   KTX Jeolla Utility Rate      215 non-null    float64
 8   KTX Donghae Utility Rate     215 non-null    float64
dtypes: float64(9)
memory usage: 16.8 KB


In [91]:
time_series1_keys = list(time_series1.keys())
time_series1_keys

['Total Passengers',
 'KTX Gyeongbu Passengers',
 'SRT Gyeongbu Passengers',
 'KTX Honam Passengers',
 'SRT Honam Passengers',
 'KTX Gyeongjeon Passengers',
 'KTX Jeolla Passengers',
 'KTX Donghae Passengers']

In [92]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
data_scaled = scaler.fit_transform(time_series1)
scaler = MinMaxScaler(feature_range=(-1, 1))
time_series1[[time_series1_keys[0]]] = scaler.fit_transform(time_series1[[time_series1_keys[0]]])


In [93]:
import torch 
import torch.nn as nn 

In [100]:
# Set your parameters
sequence_length = 216
batch_size = 10
latent_dim = 5
hidden_dim = 20
num_epochs = 200

# Define the generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.rnn = nn.GRU(latent_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.rnn = nn.GRU(1, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return torch.sigmoid(x)

# Create instances
gen = Generator()
disc = Discriminator()

# Define the loss and optimizers
criterion = nn.BCELoss()
optimizer_gen = torch.optim.Adam(gen.parameters(), lr=0.001)
optimizer_disc = torch.optim.Adam(disc.parameters(), lr=0.001)

# # Split your dataset into training and test datasets
# train_size = int(len(dataY) * 0.8)
# trainX = dataX[:train_size]
# trainY = dataY[:train_size]
# testX = dataX[train_size:]
# testY = dataY[train_size:]

# # Create DataLoader for your training data
# train_data = torch.utils.data.TensorDataset(trainX, trainY)
# train_loader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)


In [101]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

# Assuming 'feature' is the column you're interested in from your DataFrame:
seq_length = 216  # or any other window size
x, y = sliding_windows(time_series1[[time_series1_keys[0]]], seq_length)

In [102]:
# Training function
def train(gen, disc, criterion, optimizer_gen, optimizer_disc, data_loader, latent_dim, sequence_length, num_epochs):
    for epoch in range(num_epochs):
        for mini_batch, _ in data_loader:
            
            mini_batch = mini_batch.view(mini_batch.size(0), sequence_length, 1)

            # Create real and fake labels
            label_real = torch.ones(mini_batch.size(0), 1)
            label_fake = torch.zeros(mini_batch.size(0), 1)

            # Train the discriminator
            real_scores = disc(mini_batch)
            real_loss = criterion(real_scores, label_real)

            z = torch.randn(mini_batch.size(0), sequence_length, latent_dim)
            fake_data = gen(z)
            fake_scores = disc(fake_data)
            fake_loss = criterion(fake_scores, label_fake)

            d_loss = real_loss + fake_loss

            optimizer_disc.zero_grad()
            d_loss.backward()
            optimizer_disc.step()

            # Train the generator
            z = torch.randn(mini_batch.size(0), sequence_length, latent_dim)
            fake_data = gen(z)
            output = disc(fake_data)
            g_loss = criterion(output, label_real)

            optimizer_gen.zero_grad()
            g_loss.backward()
            optimizer_gen.step()

        print(f'Epoch [{epoch}/{num_epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}')

# Prediction function
def predict(gen, input_sequence, latent_dim):
    z = torch.randn(input_sequence.size(0), sequence_length, latent_dim)
    return gen(z).detach()

# Call the training function
train(gen, disc, criterion, optimizer_gen, optimizer_disc, train_loader, latent_dim, sequence_length, num_epochs)

# Use your trained model to generate a prediction
test_input = testX[0] # Replace this with your test input
test_input = test_input.view(1, sequence_length, 1) # Reshape the input
prediction = predict(gen, test_input, latent_dim)

# Now `prediction` holds the generated sequence


RuntimeError: shape '[10, 216, 1]' is invalid for input of size 320

In [98]:
print(mini_batch.shape)

(10, 216)
